In [ ]:
1+1

2

# URL

- https://www.kaggle.com/competitions/playground-series-s3e19/overview

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
le = LabelEncoder()

In [4]:
def labels_encode(df):
  df['store'] = le.fit_transform(df['store'])
  df['country'] = le.fit_transform(df['country'])
  df['product'] = le.fit_transform(df['product'])
  return df

In [5]:
def train_by_lightgbm(df):
  df_train, df_val =train_test_split(df, test_size=0.2)
  train_x = df_train[['store', 'country', 'product']]
  train_y = df_train[['num_sold']]

  val_x = df_val[['store', 'country', 'product']]
  val_y = df_val[['num_sold']]

  trains = lgb.Dataset(train_x, train_y)
  valids = lgb.Dataset(val_x, val_y)

  params = {
      "objective": "regression",
      "metrics": "mae"
  }
  model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)
  return model



In [6]:
train = labels_encode(train)
model = train_by_lightgbm(train)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001622 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13
[LightGBM] [Info] Number of data points in the train set: 109560, number of used features: 3
[LightGBM] [Info] Start training from score 165.292534
[1]	valid_0's l1: 123.589
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 111.621
[3]	valid_0's l1: 100.887
[4]	valid_0's l1: 91.2766
[5]	valid_0's l1: 82.6781
[6]	valid_0's l1: 75.0196
[7]	valid_0's l1: 68.1879
[8]	valid_0's l1: 62.1453
[9]	valid_0's l1: 56.8305
[10]	valid_0's l1: 52.1464
[11]	valid_0's l1: 48.0578
[12]	valid_0's l1: 44.4896
[13]	valid_0's l1: 41.3958
[14]	valid_0's l1: 38.7523
[15]	valid_0's l1: 36.4573
[16]	valid_0's l1: 34.507
[17]	valid_0's l1: 32.8495
[18]	valid_0's l1: 31.458
[19]	valid_0's l1: 30.2552
[20]	valid_0's l1: 29.2348


In [7]:
df_test = pd.read_csv('test.csv')
df_test = labels_encode(df_test)
predict = model.predict(df_test[['store', 'country', 'product']])
df_test['num_sold'] = predict
df_test[['id', 'num_sold']].to_csv("submission.csv", index=False)